# HW4: Your First GAN

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
# import tensorflow as tf
print (tf.__version__)

2.4.1


In [3]:
img_rows = 28
img_cols = 28
channels = 1

# Input image dimensions
img_shape = (img_rows, img_cols, channels)

# Size of the noise vector, used as input to the Generator
z_dim = 100

## Generator

In [4]:
def build_generator(img_shape, z_dim):

    model = Sequential()
### Your code here
### BEGIN
   # Fully connected layer
    model.add(Dense(128, input_dim=z_dim))

    # Leaky ReLU activation
    model.add(LeakyReLU(alpha=0.01))

    # Output layer with tanh activation
    model.add(Dense(28 * 28 * 1, activation='tanh'))

    # Reshape the Generator output to image dimensions
    model.add(Reshape(img_shape))
### END
    return model

## Discriminator

In [5]:
def build_discriminator(img_shape):

    model = Sequential()

    # Flatten the input image
    model.add(Flatten(input_shape=img_shape))

    # Fully connected layer
    model.add(Dense(128))

    # Leaky ReLU activation
    model.add(LeakyReLU(alpha=0.01))

    # Output layer with sigmoid activation
    model.add(Dense(1, activation='sigmoid'))

    return model

## Build the Model

In [6]:
def build_gan(generator, discriminator):
### Youe code
    model = Sequential()
### BEGIN
   # Combined Generator -> Discriminator model
    model.add(generator)
    model.add(discriminator)
### END


    return model

In [7]:
optimizer = Adam(lr=0.0002, beta_1=0.5)
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def build_models(use_tf2=False):

    # Build and compile the Discriminator
    discriminator = build_discriminator(img_shape)
    # Build the Generator
    generator = build_generator(img_shape, z_dim)
    gan = build_gan(generator, discriminator)


    return discriminator, generator, gan

In [8]:
# 
def accuracy(labels, logits):
    logits=tf.concat([tf.ones_like(logits)*0.5,logits],1)
    p=tf.argmax(logits, axis=1)
    p=tf.cast(p,tf.float64)
    labels=tf.reshape(labels,[-1])
    acc=tf.reduce_sum(tf.cast(tf.equal(labels, p),tf.float64))
    return acc/logits.shape[0]

# This annotation causes the function to be "compiled".
@tf.function
def train_D(model,images, labels):
    with tf.GradientTape() as tape:
      logits = model(images, training=True)
      #print(logits.shape)
      loss = bce(labels, logits)
    gradients= tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc=accuracy(labels, logits)
    return loss,acc
@tf.function
def train_G(D_model,G_model,z, labels):
    with tf.GradientTape() as tape:
      images=G_model(z)
      logits = D_model(images, training=True)
      loss = bce(labels, logits)
    gradients= tape.gradient(loss, G_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, G_model.trainable_variables))
    return loss

In [9]:
use_tf2=True
discriminator, generator, gan = build_models(use_tf2=use_tf2)


## Training

In [10]:
losses = []
accuracies = []
r_accuracies = []
f_accuracies = []
r_lose = []
f_lose = []
iteration_checkpoints = []


def train(iterations, batch_size, sample_interval,use_tf2=False):

    # Load the MNIST dataset
    (X_train, _), (_, _) = fashion_mnist.load_data()

    # Rescale [0, 255] grayscale pixel values to [-1, 1]
    X_train = X_train / 127.5 - 1.0
    X_train = np.expand_dims(X_train, axis=3)

    # Labels for real images: all ones
    real = np.ones((batch_size, 1))

    # Labels for fake images: all zeros
    fake = np.zeros((batch_size, 1))
    start = time.time()

    for iteration in range(iterations):

        # -------------------------
        #  Train the Discriminator
        # -------------------------

        # Get a random batch of real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Generate a batch of fake images
        z = np.random.normal(0, 1, (batch_size, 100))

        # Train Discriminator
        gen_imgs = generator(z)
        d_loss_real,d_acc_real = train_D(discriminator,imgs, real)
        d_loss_fake,d_acc_fake = train_D(discriminator,gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        accuracy = 0.5 * np.add(d_acc_real, d_acc_fake)

        # ---------------------
        #  Train the Generator
        # ---------------------

        # Generate a batch of fake images
        z = np.random.normal(0, 1, (batch_size, 100))

        # Train Generator
        g_loss = train_G(discriminator, generator, z, real)

        if (iteration + 1) % sample_interval == 0:

            # Save losses and accuracies so they can be plotted after training
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            r_accuracies.append(100.0 * d_acc_real)
            f_accuracies.append(100.0 * d_acc_fake)
            r_lose.append(d_loss_real)
            f_lose.append(d_loss_fake)
            iteration_checkpoints.append(iteration + 1)

            # Output training progress
            print("%d [D loss: %f, D Lose Real: %f, D Lose Fake: %f, acc.: %.2f%%, real_acc:%.2f%%, fake_acc:%.2f%%] [G loss: %f]" %
                  (iteration + 1, d_loss, d_loss_real, d_loss_fake, 100.0 * accuracy, 100.0 * d_acc_real, 100.0 * d_acc_fake, g_loss))

            # Output a sample of generated image
            sample_images(generator)
            print ('Time for interval {} is {} sec'.format((iteration + 1)//sample_interval, time.time()-start))
            start = time.time()


In [11]:
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):

    # Sample random noise
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # Generate images from random noise
    gen_imgs = generator.predict(z)

    # Rescale image pixel values to [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Set image grid
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(4, 4),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # Output a grid of images
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    plt.show()

## Train the GAN and Inspect Output

Note that the `'Discrepancy between trainable weights and collected trainable'` warning from Keras is expected. It is by design: The Generator's trainable parameters are intentionally held constant during Discriminator training, and vice versa.

In [12]:
sample_images(generator)

InternalError:  Blas GEMM launch failed : a.shape=(16, 100), b.shape=(100, 128), m=16, n=128, k=100
	 [[node sequential_1/dense_2/MatMul (defined at <ipython-input-11-f9175d688084>:7) ]] [Op:__inference_predict_function_268]

Function call stack:
predict_function


In [ ]:
# Set hyperparameters
iterations = 20000
batch_size = 128
sample_interval = 100
# Build models
# Train the GAN for the specified number of iterations
train(iterations, batch_size, sample_interval,use_tf2=use_tf2)

In [ ]:
losses = np.array(losses)

# Plot training losses for Discriminator and Generator
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, losses.T[0], label="Discriminator loss")
plt.plot(iteration_checkpoints, losses.T[1], label="Generator loss")
plt.plot(iteration_checkpoints, f_lose, label="Discriminator lose of fake")
plt.plot(iteration_checkpoints, r_lose, label="Discriminator lose of real")


plt.xticks(iteration_checkpoints, rotation=90)

plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracies = np.array(accuracies)

# Plot Discriminator accuracy
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, accuracies, label="Discriminator accuracy")
plt.plot(iteration_checkpoints, f_accuracies, label="Discriminator accuracy of fake")
plt.plot(iteration_checkpoints, r_accuracies, label="Discriminator accuracy of real")

plt.xticks(iteration_checkpoints, rotation=90)
plt.yticks(range(0, 100, 5))

plt.title("Discriminator Accuracy")
plt.xlabel("Iteration")
plt.ylabel("Accuracy (%)")
plt.legend()

----

In [ ]:
def evaluate(n_sample=100, use_tf2=False):
    # Generate a batch of fake images
    z = np.random.normal(0, 1, (n_sample, 100))

    # Train Discriminator
    if use_tf2:
        gen_imgs = generator(z)
    else: 
        gen_imgs = generator.predict(z)
    real = np.ones((n_sample, 1))
    fake = np.zeros((n_sample, 1))
    a=accuracy(real, discriminator(gen_imgs))
    print ("%d%% generated images are predicted as real"%int(a*100))
    idx=np.argwhere(tf.reshape(discriminator(gen_imgs),-1)>0.5).reshape(-1)
    print (idx)
    for i in idx:
      sel_imgs=gen_imgs[i]
      print(discriminator(gen_imgs[i:i+1]))
      plt.imshow(sel_imgs,cmap='gray')
      plt.show()
    
evaluate(n_sample=100,use_tf2=use_tf2)